In [1]:
import pandas as pd
import json
import requests

In [2]:
PATH = "data/validation.xlsx"
annotations = pd.read_excel(PATH, names=["id_image","caption"])

In [3]:
PATH = "./results/val_M038_E.csv"
results = pd.read_csv(PATH)
results = results.drop_duplicates()

In [4]:
json_annotations = {
    'info': {
        'description':''
    },
    'images': [],
    'licenses': [],
    'type': 'captions',
    'annotations': []
}

In [5]:
def dicts_equal(d1,d2):
    """ return True if all keys and values are the same """
    return all(k in d2 and d1[k] == d2[k]
               for k in d1) \
        and all(k in d1 and d1[k] == d2[k]
               for k in d2)

In [6]:
c=0
for annotation in annotations.iterrows():
    ann = {
        'image_id': annotation[1]['id_image'],
        'id': c,
        'caption': annotation[1]['caption']
    }
    img = {
        'license':3,
        'url': '',
        'file_name': '',
        'id': annotation[1]['id_image'],
        'width': 640,
        'date_captured': '',
        'height': 360,
    }
    for i in json_annotations['images']:
        if not dicts_equal(i,img):
            json_annotations['images'].append(img)
            break
    if len(json_annotations['images'])==0:
        json_annotations['images'].append(img)
    c+=1
    json_annotations['annotations'].append(ann)

In [7]:
json_results = []

In [8]:
for result in results.iterrows():
    res = {
        'image_id': result[1]['id_image'],
        'caption': result[1]['caption']
    }
    json_results.append(res)

In [9]:
with open("./coco-caption/annotations/captions_val2015.json",'w') as f:
    json.dump(json_annotations, f)

In [10]:
with open("./coco-caption/results/captions_val2015_fakecap_results.json",'w') as f:
    json.dump(json_results, f)

## Post-traducción

In [4]:
malos_acentos = ['Ã¡', 'Ã©', 'Ã*', 'Ã³', 'Ãº', 'Ã', 'Ã‰', 'Ã', 'Ã“', 'Ãš', 'Ã±', 'Ã‘', 'Âº', 'Âª', 'Â¿','Á±','Á­','Á�','贸','単']
buenos_acentos = ['á', 'é', 'í', 'ó', 'ú', 'Á', 'É', 'Í', 'Ó', 'Ú', 'ñ', 'Ñ', 'º', 'ª', '¿','ñ','í','Á','ó','ñ']

In [ ]:
c=0
for index,result in results.iterrows():
    caption = result['caption']
    data = [
        ('auth_key', '3043e601-5322-a0bf-45b0-12d05f1b1073'),
        ('text', '%s'%caption),
        ('source_lang', 'EN'),
        ('target_lang', 'ES'),
    ]
    response = requests.post('https://api.deepl.com/v2/translate?split_sentences=0', data=data)
    translation = json.loads(response.text)['translations'][0]['text']
    c+=1
    print(c)
    results.loc[index,'caption'] = translation

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119


In [22]:
PATH = "./results/val_M032_E.csv"
results = results.to_csv(PATH,index=False, encoding = 'utf-8')